In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [2]:
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display
import os

In [3]:
from datetime import datetime

시간에 대응할 수 있는 변수가 존재하는 데이터셋이라면 시계열이 있다고 본다.

# 테이블 집합에서 시계열 데이터 집합 개선하기

시계열의 대표적 예는 SQL DB에 저장된 상태 및 이벤트형 데이터로부터 추출된 데이터이다. 다음과 같이 시계열 분석에 도움이 되는 요소가 추가되어있다고 가정해보자.
1. 시간이 지나면서 바뀌는 메일 수신자의 반응 : 메일을 읽었는가 읽지 않았는가
2. 멤버십 내역 : 회원이 회원 자격을 상실한 기간이 있었는가
3. 거래 내역 : 소비자가 특정 항목을 언제 구매할지 예측할 수 있는가

위와 같은 것들은 몇가지 시계열 기법으로 살펴볼 수 있다.
1. 메일에 응답한 회원과 회원별 응답 시간의 관계를 히스토그램으로 만들어서 메일 수신에 따른 회원의 피로감을 알 수 있다.
2. 기부 예측은 시계열 예측 문제로 전환할 수 있다.
3.  중요한 상황에서 회원의 행동 자취에 대한 전형적인 패턴의 유무를 검사해볼 수 있다. ex)이메일 세 개를 연달아 지움

보통의 DB 스키마를 설계할 때 시계열 분석을 고려하지 않는것이 보통이다. 이러한 경우 서로 다른 테이블과 자료로부터 시계열을 수집하고 구성할 필요가 있다.

## 시계열 데이터 집합 조립
다음과 같은 데이터가 생성되어 있다고 하자 (인덱스 무시)

In [4]:
YearJoined = pd.read_csv("https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/year_joined.csv")
emails = pd.read_csv("https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/emails.csv")
donations = pd.read_csv("https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/donations.csv")


In [5]:
display(YearJoined.head())
display(emails.head())
display(donations.head())

,user,userStats,yearJoined
0,0,silver,2014
1,1,silver,2015
2,2,silver,2016
3,3,bronze,2018
4,4,silver,2018


,emailsOpened,user,week
0,3.0,1.0,2015-06-29 00:00:00
1,2.0,1.0,2015-07-13 00:00:00
2,2.0,1.0,2015-07-20 00:00:00
3,3.0,1.0,2015-07-27 00:00:00
4,1.0,1.0,2015-08-03 00:00:00


,amount,timestamp,user
0,25.0,2017-11-12 11:13:44,0.0
1,50.0,2015-08-25 19:01:45,0.0
2,25.0,2015-03-26 12:03:47,0.0
3,50.0,2016-07-06 12:24:55,0.0
4,50.0,2016-05-11 18:13:04,1.0


In [6]:
print(YearJoined.shape, emails.shape, donations.shape)

(1000, 3) (25488, 3) (2676, 3)


In [7]:
# 1000명의 모든 회원이 단 하나의 상태만을 가진다. 
# 그러므로 YearJoined는 각 회원이 가입한 연도와 더불어 현재 또는 가입 당시의 상태를 나타낼 가능성이 높다.
YearJoined.groupby('user').count().groupby('userStats').count()

,yearJoined
userStats,
1,1000


### 사전관찰(look-ahead)
데이터를 통해 실제로 알아야하는 시점보다 더 일찍 미래에 대한 사실을 발견하는 방법. 어떨게든 미래에 일어날 일에 대한 정보가 모델에서 시간을 거슬러 전파되어 모델의 초기 동작에 영향을 주는 방법이다.<br>
과거 데이터 분석에 회원의 현재 상태를 적용하면 그때는 알 수 없는 시계열 모델에 무언가를 입력해주기 때문에 사전관찰이라고 볼 수 있다.

### email 데이터
week열의 경우 데이터가 매주 특정 시점 또는 매주의 기간에 대한 기록임을 알 수 있다. 일부 중요한 특징들을 평가해봐야하는데 예를 들어 어떤 방식으로 주(week)를 시간(time)으로 표현했는지와 같은 것이다. 일반적으로 산업에서 사용하는 방법과 비교하여 이상하게 주가 나눠져 있다면 어떤 방식을 사용했는지 알아야한다. 대개 일요일\~토요일 혹은 월요일\~일요일로 주를 정하는 것이 좋다. 1월 1일을 한 주의 시작으로 정해서는 안된다.<br>
어떤 주에 null이 있는지 역시 확인해야한다. 즉 특정 회원이 단 한번도 이메일을 열람하지 않은 주가 있는지에 대한 것이다. 시간 지향적인 모델을 만든다면 이런 사실이 중요한데 아무런 일도 발생하지 않은 주 역시 데이터의 일부이기 때문이다.

In [8]:
# null이 아예 발생하지 않았거나 모든 회원이 이메일을 열람했다는 이벤트가 적어도 하나 존재한다는 두 가지 가능성이 있다.
# 이메일을 열람하도록 사람들을 유도하는 것은 어렵기 때문에 모든 회원이 매주 적어도 한 번은 이메일을 열람한다는 것은 가능성이 낮다.
emails[emails.emailsOpened < 1]

,emailsOpened,user,week


In [9]:
emails.emailsOpened.value_counts()

3.0    15350
1.0     5070
2.0     5068
Name: emailsOpened, dtype: int64

In [10]:
# 특정 한 회원에 대한 기록을 살펴봄으로써 둘 중 무엇인지 판단할 수 있다.
# 일부 주가 누락되었음 확인(2017.12.25)
emails[emails.user == 998]

,emailsOpened,user,week
25464,1.0,998.0,2017-12-04 00:00:00
25465,3.0,998.0,2017-12-11 00:00:00
25466,3.0,998.0,2017-12-18 00:00:00
25467,3.0,998.0,2018-01-01 00:00:00
25468,3.0,998.0,2018-01-08 00:00:00
25469,2.0,998.0,2018-01-15 00:00:00
25470,3.0,998.0,2018-01-22 00:00:00
25471,2.0,998.0,2018-01-29 00:00:00
25472,3.0,998.0,2018-02-05 00:00:00
25473,3.0,998.0,2018-02-12 00:00:00


In [11]:
# 다음 두 개를 비교해 확인할 수도 있다. (모든 주가 포함되었다면 24개가 아니라 26개가 있어야함)
print(emails[emails.user == 998].shape)
print((datetime.strptime(np.max(emails[emails.user == 998].week),'%Y-%m-%d %H:%M:%S') - datetime.strptime(np.min(emails[emails.user == 998].week),'%Y-%m-%d %H:%M:%S')).days/7)

(24, 3)
25.0


In [12]:
def email_response(member):
    maximum = datetime.strptime(np.max(emails[emails.user == member].week),'%Y-%m-%d %H:%M:%S')
    minimum = datetime.strptime(np.min(emails[emails.user == member].week),'%Y-%m-%d %H:%M:%S')
    return (maximum - minimum).days/7 - 24 + 1

In [13]:
pd.Series(np.unique(emails.user)).map(email_response)

0      129.0
1      -16.0
2       28.0
3       54.0
4       74.0
       ...  
534    -23.0
535     -2.0
536     14.0
537     67.0
538      2.0
Length: 539, dtype: float64

누락된 주가 있기 때문에 그부분을 채워 완전한 데이터셋을 완성하도록 한다. 코드를 직접 작성해도 되지만 판다스의 색인 기능을 이용해 해결하기로 한다.

In [14]:
# 모든 회원과 주에 대한 조합을 생성 (cartesian product를 생성)
complete_idx = pd.MultiIndex.from_product((set(emails.week), set(emails.user)))

# 재색인
all_email = emails.set_index(['week','user']).reindex(complete_idx,fill_value=0).reset_index()
all_email.columns = ['week', 'user', 'emailsOpened']

In [15]:
# 다시한번 998번 회원을 확인
all_email[all_email.user == 998].sort_values('week')


,week,user,emailsOpened
63062,2015-02-09 00:00:00,998.0,0.0
40424,2015-02-16 00:00:00,998.0,0.0
4850,2015-02-23 00:00:00,998.0,0.0
26949,2015-03-02 00:00:00,998.0,0.0
51743,2015-03-09 00:00:00,998.0,0.0
...,...,...,...
11318,2018-04-30 00:00:00,998.0,3.0
67374,2018-05-07 00:00:00,998.0,3.0
19403,2018-05-14 00:00:00,998.0,3.0
56594,2018-05-21 00:00:00,998.0,3.0


In [16]:
all_email.shape

(93247, 3)

0이 많이 등장하는데 회원 가입 전단계라 회원의 이메일 주소가 목록에 포함되지 않아 발생하는 값이다. 회원의 최초 이메일 수신 시점에 대한 정확한 데이터가 없으므로 주어진 데이터로부터 이를 도출해야한다.

In [17]:
cutoff_dates = emails.groupby('user').week.agg(['min','max']).reset_index()

In [18]:
cutoff_dates.head()

,user,min,max
0,1.0,2015-06-29 00:00:00,2018-05-28 00:00:00
1,3.0,2018-03-05 00:00:00,2018-04-23 00:00:00
2,5.0,2017-06-05 00:00:00,2018-05-28 00:00:00
3,6.0,2016-12-05 00:00:00,2018-05-28 00:00:00
4,9.0,2016-07-18 00:00:00,2018-05-28 00:00:00


In [19]:
# 위 데이터프레임을 이용해 단순히 0이 아닌 값이 나타나기 전의 값은 아예 삭제하도록 한다.
for i in range(len(cutoff_dates)):
    user = cutoff_dates.iloc[i,0]
    minimum = cutoff_dates.iloc[i,1]
    maximum = cutoff_dates.iloc[i,2]
    
    cond1 = all_email['week']<minimum
    cond2 = all_email['week']>maximum
    cond3 = all_email['user'] == user
    
    cond1_3 = cond1&cond3
    cond2_3 = cond2&cond3
    
    all_email.drop(all_email[cond1_3|cond2_3].index,inplace=True)
    

In [20]:
all_email.shape

(31836, 3)

시간을 인식할 수 있는 형태, 특히 타임스탬프의 생성 방식과 그 의미에 따라 데이터를 저장해야한다. 

이메일에 대한 반응과 기부의 상관관계를 알고 싶다면 다음과 같은 방식을 고려해볼 수 있다.
- 순간 순간 기록된 데이터를 주 단위로 기부금을 취합한다면 이메일 데이터와 기간 단위의 비교가 가능해진다. 따라서 회원의 이메일 응답과 기부가 어떤 식으로 연관이 있는지에 대해 합리적으로 조사할 수 있다.
- 전 주의 열람된 이메일 개수(EmailOpened)를 주어진 주의 기부금 예측 변수로 취급할 수 있다. 여기서 전 주를 사용하는 이유는 EailsOpened이 해당 주에 대한 요약 통계이기 때문이다. 가령 수요일의 기부를 예측하고자 한다면 EmailsOpened가 월요일부터 일요일까지 일어난 이메일 열람 행동을 요약한다고 가정해보자. 이 둘의 대상 주가 같다면 특정 회원이 기부를 한 다음에 어떤 일을 할지에 대한 정보를 잠재적으로 얻게 된다. 기부를 한 다음에 오는 금요일에 기부자가 이메일을 열람할지에 대한 경우를 예로 들 수 있다.

## 발견된 시계열 구성
이메일과 기부 데이터를 연관시키고자 하는데 기부 데이터를 다운샘플링하여 주간 시계열로 바꿀 수 있다.

In [21]:
donations['timestamp'] = pd.to_datetime(donations['timestamp'])
donations.set_index('timestamp',inplace=True)

In [22]:
display(donations.head())

,amount,user
timestamp,,
2017-11-12 11:13:44,25.0,0.0
2015-08-25 19:01:45,50.0,0.0
2015-03-26 12:03:47,25.0,0.0
2016-07-06 12:24:55,50.0,0.0
2016-05-11 18:13:04,50.0,1.0


In [23]:
# resample은 고정된 빈도로 시계열을 변환할 수 있다. 당연히 누락값이 발생할 수 있다.
# 각 회원의 부분집합에 대한 데이터프레임을 구하여 주단위로 그룹화하여 기부금 총액을 구하고 기부가 없는 주는 삭제한다.
agg_don = donations.groupby('user').apply(lambda df:df['amount'].resample("W-MON").sum().dropna())
agg_don

user   timestamp 
0.0    2015-03-30      25.0
       2015-04-06       0.0
       2015-04-13       0.0
       2015-04-20       0.0
       2015-04-27       0.0
                      ...  
995.0  2017-09-11       0.0
       2017-09-18       0.0
       2017-09-25       0.0
       2017-10-02    1000.0
998.0  2018-01-08      50.0
Name: amount, Length: 32352, dtype: float64

In [24]:
# 동일한 빈도로 샘플링된 기부와 이메일 정보를 하나로 합쳐볼 수 있다.
# 모든 주를 같은 요일로 간단하게 고정하여 각 회원의 정보에 접근하여 회원별 이메일 및 기부에 대한 데이터프레임을 통합한다.
display(all_email.head())

,week,user,emailsOpened
0,2017-12-04 00:00:00,1.0,3.0
2,2017-12-04 00:00:00,5.0,1.0
3,2017-12-04 00:00:00,6.0,2.0
4,2017-12-04 00:00:00,9.0,3.0
6,2017-12-04 00:00:00,14.0,2.0


In [25]:
merged_df = pd.DataFrame()
for member, member_email in all_email.groupby('user'):
    # 특정 회원의 기부 데이터
    member_donations = agg_don[agg_don.index.get_level_values('user')==member]
    # 재색인
    member_donations = member_donations.droplevel(0)
    member_email.set_index('week',inplace=True)
    member_email.sort_index(inplace = True)
    
    # 병합
    df = pd.merge(member_email, member_donations, how='left',
                 left_index = True, right_index=True)
    df.fillna(0)
    
    merged_df = merged_df.append(df.reset_index()[['user', 'week', 'emailsOpened', 'amount']])

In [31]:
merged_df.head()

,user,week,emailsOpened,amount
0,1.0,2015-06-29 00:00:00,3.0,NaN
1,1.0,2015-07-06 00:00:00,0.0,NaN
2,1.0,2015-07-13 00:00:00,2.0,NaN
3,1.0,2015-07-20 00:00:00,2.0,NaN
4,1.0,2015-07-27 00:00:00,3.0,NaN


이메일 응답을 기부 관련 '상태' 변수로 다룰 수도 있지만, 사전관찰을 방지하려면 전 주의 상태를 이월해야한다. 이메일 응답 방식에 회원의 다음 기부를 예측하는 모델을 만들고자 할 때 시간에 따른 이메일 열람의 행동 패턴을 하나의 지표로 고려해야한다. 그러면 한 주의 기부금을 기부가 발생하기 전 주의 이메일 응답 방식에 따라서 정렬해야 한다.(shift 연산자 사용)

In [33]:
df = merged_df[merged_df['user']==998]
df['target'] = df['amount'].shift(1)
df = df.fillna(0)
df.head(8)

,user,week,emailsOpened,amount,target
0,998.0,2017-12-04,1.0,0.0,0.0
1,998.0,2017-12-11,3.0,0.0,0.0
2,998.0,2017-12-18,3.0,0.0,0.0
3,998.0,2017-12-25,0.0,0.0,0.0
4,998.0,2018-01-01,3.0,0.0,0.0
5,998.0,2018-01-08,3.0,50.0,0.0
6,998.0,2018-01-15,2.0,0.0,50.0
7,998.0,2018-01-22,3.0,0.0,0.0


### 시계열에 특화된 데이터 재구성 기법
- 해결하고자 하는 문제에 맞는 형태로 데이터의 간격을 교정한다.(종종 필요 이상으로 구체적인 시간 정보를 제공하기도함)
- 사전관찰을 피하기 위해 가용 데이터를 생산하는 타임스탬프를 데이터에 사용하지 않는 방법을 이해한다.
- 아무 일도 일어나지 않았다하더라도 관련된 모든 기간을 기록한다. 합계가 0일지라도, 다른 정보들처럼 유익한 정보가 된다.


# 타임스탬프의 문제점
## 무엇에 대한 타임스탬프인가 정의
타임스탬프의 사건의 시간이 무엇에 의해 정해졌는지 최대한 알아보는 일이 중요하다.(명료성) 예를 들어 오프라인에서 모바일 앱 사용 후 휴대전화가 온라인으로 바뀔 때 서버로 로그 정보가 전송되는데 이 때 기록된 타임스탬프가 사용자의 작업 시작 시점, 앱이 기록한 작업의 시점, 데이터가 서버에서 앱으로 다운로드된 가장 마지막 시점 등 여러 의미를 가질 수 있다. 혹은 시간 자체를 무시할  수 있는 옵션에 대한 인터페이스가 제공될 수도 있다.

이때 타임스탬프의 의미를 추론할 수 있는지 여부를 파악하기 위해 몇 가지 조사를 해볼 수 있다.
- 타임스탬프 의미에 대해 초기 가설을 세워보고 여러 사용자 데이터를 통해 동일 패턴이 유지되는지, 이례적 패턴이 나타나는지 확인해본다.
- 타임스탬프 의미에 대한 가설을 시험해본다.
    - 타임스탬프가 현지 시간을 의미하는지 세계 표준 시간을 의미하는지
    - 시간이 사용자의 행동을 반영하는지 아니면 연결과 같은 외부 제약을 반영하는지

In [ ]:
# 절대적 시간(세계 표준시)에 기반한 타임스탬프를 활용하면 기록된 타임스탬프 간 시차를 계산할 수 있다.
df['dt'] = df['time'] - df['time'].shift(-1)

# dt가 사용자 행동에 대한 것이라면 네트워크 연결이나 소프트웨어에 관련한 행동보다 주기적인 성향이 더 강할 가능성이 높다.
# dt가 네트워크 연결에 대한 것이면 서버가 뢀성될 가능성이 높은 시점을 찾을 수도 있다.

## 의미있는 시간 규모
- 인지편향을 줄이고자 데이터의 시간을 일 단위에서 주 단위로 바꾸는 것을 고려해볼 수 있다.
- 사람의 혈당이 빠르게 변하는 속도에는 한계가 있는데 일련의 혈당 수치가 초 단위로 기록된다면 이를 개별적으로 다루기보다 평균으로 취급하는 것이 좋다.